<a href="https://colab.research.google.com/github/Silver0x10/VideoPrediction_MovingMNIST/blob/main/notebooks/VideoPredictionMovingMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
setup_already_done = False

In [ ]:
import sys
import os

In [ ]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

In [ ]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")):
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..

    %pip install -qr requirements.txt  --quiet
    setup_already_done = True

In [ ]:
!nvidia-smi

In [ ]:
!pwd
# %cd VideoPrediction_MovingMNIST/

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader,random_split

import lightning.pytorch as pl
# from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger

import numpy as np

# import pylab as plt
import matplotlib.pyplot as plt

from src.utils import *

import wandb

In [ ]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type

In [ ]:
wandb.login()

# Dataset

In [ ]:
from src.MovingMNIST import MovingMNIST
from src.parameters import shared_params

In [ ]:
dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")

In [ ]:
train_set, validation_set, test_set = random_split(dataset,[8000, 1000, 1000],
                                                   generator=torch.Generator().manual_seed(42))

## Reference Sample

In [ ]:
sequence_test = test_set[42]
complete_sequence = torch.cat((sequence_test['frames'], sequence_test['y']))
dataset.visualize_given_frames_as_gif(complete_sequence)

In [ ]:
show_video_line(complete_sequence, 20)

In [ ]:
print("Input:")
input_frames = sequence_test['frames']
dataset.visualize_given_frames_as_gif(input_frames)

In [ ]:
print("Ground Truth:")
gt_frames = sequence_test['y']
dataset.visualize_given_frames_as_gif(gt_frames)

# 1) SimpleLSTM

In [ ]:
from src.simpleLSTM import SimpleLSTM
from src.parameters import ParamsSimpleLSTM

params_simpleLSTM = ParamsSimpleLSTM()
model_simpleLSTM = SimpleLSTM(params_simpleLSTM)

In [ ]:
params_simpleLSTM.training_epochs = 50
params_simpleLSTM.training_epochs

## Training + Testing

In [ ]:
training_dataloader = DataLoader(train_set, batch_size = params_simpleLSTM.batch_size)
validation_dataloader = DataLoader(validation_set, batch_size = params_simpleLSTM.batch_size)
test_dataloader = DataLoader(test_set, batch_size = params_simpleLSTM.batch_size)

In [ ]:
wandb_logger = WandbLogger(project='DeepLearning', name='SimpleLSTM_50epochs', log_model=True)

In [ ]:
wandb_logger.experiment.config['layers'] = str(model_simpleLSTM)

for p in vars(params_simpleLSTM).keys():
    wandb_logger.experiment.config[p] = vars(params_simpleLSTM)[p]

In [ ]:
trainer_simpleLSTM = pl.Trainer(max_epochs=params_simpleLSTM.training_epochs,
                                accelerator=device.type,
                                logger=wandb_logger,
                                # deterministic=True,
                                # fast_dev_run=1,
                                # overfit_batches=1, # to test if the model can learn
                                detect_anomaly=True)

In [ ]:
trainer_simpleLSTM.fit(model=model_simpleLSTM, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
results_simpleLSTM = trainer_simpleLSTM.test(model_simpleLSTM, dataloaders=test_dataloader)

## Visualization Example

In [ ]:
# checkpoint_path = "lightning_logs/vo3td60s/checkpoints/epoch=9-step=80000.ckpt"
# # checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict(checkpoint['state_dict'])
# model.eval()

In [ ]:
print("Prediction:")
pred_simpleLSTM, _ = model_simpleLSTM(input_frames.unsqueeze(0))
pred_simpleLSTM = pred_simpleLSTM.detach().squeeze(0)
dataset.visualize_given_frames_as_gif(pred_simpleLSTM)

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(14, 28))
ax[0,0].set_title('Ground Truth')
ax[0,1].set_title('Prediction')
ax[0,2].set_title('Difference')

for i in range(gt_frames.shape[0]):
    gt_i = gt_frames[i, :, :]
    pred_i = pred_simpleLSTM[i, :, :]
    ax[i,0].imshow(gt_i, cmap='gray')
    ax[i,1].imshow(pred_i, cmap='gray')
    ax[i,2].imshow(gt_i - pred_i, cmap='gray')

fig.savefig('out/simpleLSTM_comparison.png')
wandb.log({"example": wandb.Image("out/simpleLSTM_comparison.png")})

fig.show()

## SimpleLSTM Finish

In [ ]:
wandb.finish()

In [ ]:
# !zip -r ../logs lightning_logs/ # Remember to save weights (if needed)

# 2) ConvLSTM

In [ ]:
from src.convLSTM import *
from src.parameters import ParamsConvLSTM

params_convLSTM = ParamsConvLSTM()
model_convLSTM = PlEncoderDecoder(k_s =3, Batch_size =1)#params_convLSTM.batch_size)

## Training + Testing

In [ ]:
training_dataloader = DataLoader(train_set, batch_size = params_convLSTM.batch_size)
validation_dataloader = DataLoader(validation_set, batch_size = params_convLSTM.batch_size)
test_dataloader = DataLoader(test_set, batch_size = params_convLSTM.batch_size)

In [ ]:
wandb_logger = WandbLogger(project='DeepLearning', name='ConvLSTM', log_model=True)

In [ ]:
wandb_logger.experiment.config['layers'] = str(model_convLSTM)

for p in vars(params_convLSTM).keys():
    wandb_logger.experiment.config[p] = vars(params_convLSTM)[p]

In [ ]:
trainer_convLSTM= pl.Trainer(max_epochs=params_convLSTM.training_epochs,
                             accelerator=device.type,
                             logger=wandb_logger,
                            #  deterministic=True,
                            #  fast_dev_run=1,
                            # overfit_batches=1, # to test if the model can learn
                            detect_anomaly=True)

In [ ]:
trainer_convLSTM.fit(model=model_convLSTM, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
results_convLSTM = trainer_convLSTM.test(model_convLSTM, dataloaders=test_dataloader)

## Reference Prediction:

In [ ]:
# Load weights if needed

# checkpoint_path = "lightning_logs/vo3td60s/checkpoints/epoch=9-step=80000.ckpt"
# # checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict(checkpoint['state_dict'])
# model.eval()

In [ ]:
print("Prediction:")
pred_convLSTM, _ = model_convLSTM(input_frames.unsqueeze(0))
pred_convLSTM = pred_convLSTM.detach().squeeze(0).cpu()
dataset.visualize_given_frames_as_gif(pred_convLSTM)

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(14, 28))
ax[0,0].set_title('Ground Truth')
ax[0,1].set_title('Prediction')
ax[0,2].set_title('Difference')

for i in range(gt_frames.shape[0]):
    gt_i = gt_frames[i, :, :]
    pred_i = pred_convLSTM[i, :, :].cpu()
    ax[i,0].imshow(gt_i, cmap='gray')
    ax[i,1].imshow(pred_i, cmap='gray')
    ax[i,2].imshow(gt_i - pred_i, cmap='gray')

fig.savefig('out/convLSTM_comparison.png')
wandb.log({"example": wandb.Image("out/convLSTM_comparison.png")})

fig.show()

## ConvLSTM Finish

In [ ]:
wandb.finish()

In [ ]:
# !zip -r ../logs lightning_logs/ # Remember to save weights (if needed)

# 3) ConvTAU

In [ ]:
from src.ConvTAU import ConvTAU
from src.parameters import ParamsConvTAU

params_convTAU = ParamsConvTAU()
model_convTAU = ConvTAU(params_convTAU)

## Training + Testing

In [ ]:
training_dataloader = DataLoader(train_set, batch_size = params_convTAU.batch_size)
validation_dataloader = DataLoader(validation_set, batch_size = params_convTAU.batch_size)
test_dataloader = DataLoader(test_set, batch_size = params_convTAU.batch_size)

In [ ]:
wandb_logger = WandbLogger(project='DeepLearning', name='ConvTAU_original_50epochs_', log_model=True)

In [ ]:
wandb_logger.experiment.config['layers'] = str(model_convTAU)

for p in vars(params_convTAU).keys():
    wandb_logger.experiment.config[p] = vars(params_convTAU)[p]

In [ ]:
trainer_convTAU = pl.Trainer(max_epochs=params_convTAU.training_epochs,
                             accelerator=device.type,
                             logger=wandb_logger,
                            #  deterministic=True,
                            #  fast_dev_run=5,
                            #  overfit_batches=1, # to test if the model can learn
                             detect_anomaly=True)

In [ ]:
trainer_convTAU.fit(model=model_convTAU, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

In [ ]:
results_convTAU = trainer_convTAU.test(model_convTAU, dataloaders=test_dataloader)

## Visualization Example

In [ ]:
# checkpoint_path = "lightning_logs/vo3td60s/checkpoints/epoch=9-step=80000.ckpt"
# # checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
# checkpoint = torch.load(checkpoint_path)

# model_convTAU.load_state_dict(torch.load('out/convTAU_weights.pt'))
# model_convTAU.eval()


In [ ]:
print("Prediction:")
pred_convTAU = model_convTAU.single_prediction(input_frames).detach().squeeze(1)
dataset.visualize_given_frames_as_gif(pred_convTAU)

In [ ]:
show_video_line(pred_convTAU, 10)

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(14, 28))
ax[0,0].set_title('Ground Truth')
ax[0,1].set_title('Prediction')
ax[0,2].set_title('Difference')

for i in range(gt_frames.shape[0]):
    gt_i = gt_frames[i, :, :]
    pred_i = pred_convTAU[i, :, :]
    ax[i,0].imshow(gt_i, cmap='gray')
    ax[i,1].imshow(pred_i, cmap='gray')
    ax[i,2].imshow(gt_i - pred_i, cmap='gray')

fig.savefig('out/convTAU_comparison.png')
wandb.log({"example": wandb.Image("out/convTAU_comparison.png")})

fig.show()

## ConvTAU Finish

In [ ]:
torch.save(model_convTAU.state_dict(), 'out/convTAU_weights.pt')

In [ ]:
wandb.finish()

# Extra / Trash

In [ ]:
!zip -r ../out out/ # Remember to save/download weights and outputs (if needed)

In [ ]:
# import gc
# print(torch.cuda.list_gpu_processes())
# gc.collect()
# torch.cuda.empty_cache()